# Laplace equation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import numba

In [ ]:
N = 100
X = 2.0
thres = 0.05
center = np.array((N // 2, N // 2))
delta = X / N
Conv = 1.0e-6

In [ ]:
@numba.jit
def calc_phi_at(i, j, phi: np.ndarray):
    return 0.25*(phi[i+1, j]+phi[i-1, j]+phi[i, j+1]+phi[i, j-1])

@numba.jit
def init_phi(phi):
    for i in range(N):
        for j in range(N):
            r2 = ((center[0] - i)**2 + (center[1] - j)**2)*(delta**2)
            if (1 - thres)**2 <= r2 <= 1:
                x_ = i - center[0]
                y_ = j - center[1]
                if x_ == 0:
                    theta = np.pi/2 if y_ >= 0 else -np.pi/2
                else:
                    tan = y_ / x_
                    if x_ >= 0:
                        theta = np.arctan(tan)
                    else:
                        theta = np.arctan(tan) + np.pi
                phi[i, j] = np.cos(3*theta)

@numba.jit
def main_loop():
    #phi = np.zeros((N, N), dtype=numba.float32)
    phi = np.zeros((N, N))
    init_phi(phi)
    MaxPhi_list = []
    loop = 0
    MaxPhi = 1.0e-10
    while True:
        if loop%1000 == 0:
            print(loop, MaxPhi)        

        MaxErr = CurErr = 0
        for i in range(N):
            for j in range(N):
                r2 = ((center[0] - i)**2 + (center[1] - j)**2)*(delta**2)
                if r2 >= (1 - thres)**2:
                    continue
                Prev_phi = phi[i, j]
                phi[i, j] = calc_phi_at(i, j, phi)

                if MaxPhi < abs(phi[i, j]):
                    MaxPhi = phi[i, j]

                CurErr = abs(phi[i, j] - Prev_phi) / MaxPhi

                if MaxErr < CurErr:
                    MaxErr = CurErr
        MaxPhi_list.append(MaxErr)
        loop += 1
        if MaxErr <= Conv:
            return phi, MaxPhi_list

In [ ]:
%%time

phi, MaxPhi_list = main_loop()

In [ ]:
with open('phi.pkl', 'wb') as fout:
    pickle.dump(phi, fout)

In [ ]:
fig, ax = plt.subplots()
ax.set_yscale('log')
plt.plot(range(len(MaxPhi_list)), MaxPhi_list)
plt.show()

## Visualization of the electrostatic potential

In [ ]:
fig, ax = plt.subplots()
xs, ys = np.meshgrid(np.arange(N), np.arange(N))
zs = phi[xs, ys]
xs_, ys_ = np.meshgrid(np.arange(N)*delta, np.arange(N)*delta)
im = ax.pcolormesh(xs_, ys_, zs, vmin=np.min(phi), vmax=np.max(phi), cmap='rainbow') # or jet
fig.colorbar(im, ax=ax)
ax.set_aspect('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})
ax.plot_surface(xs_, ys_, zs, vmin=zs.min(), cmap='rainbow')
fig.colorbar(im, ax=ax)
plt.xlabel('x')
plt.ylabel('y')
plt.show()